#### **How my OP codes look**

In [2]:
%matplotlib inline

import sys
import xarray as xr
import numpy as np
import os
import math
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ErrorCode, ParcelsRandom

sys.path.append('/Users/jvalenti/MOAD/analysis-jose/notebooks/parcels')
from Kernels_biofilm import DeleteParticle, Buoyancy, AdvectionRK4_3D, Stokes_drift, Beaching, Unbeaching
from OP_functions_biofilm import *

In [3]:
# Define paths
local = 1 #Set to 0 when working on server
paths = path(local)

In [5]:
#path_NEMO = make_prefix(datetime(2018, 12, 23), paths['NEMO']) #IN RESULTS2 :(
coord=xr.open_dataset(paths['coords'],decode_times=False)
#WW3 = xr.open_dataset(get_WW3_path(datetime(2018, 12, 23))) 
batt=xr.open_dataset(paths['mask'],decode_times=False)

Uniform distribution of points

In [6]:
#clat,clon = p_unidist(coord.gphif[0,:,:],coord.glamf[0,:,:],batt.mbathy[0,:,:],10,10) it takes 15 minutes to run.

with open('clat2.txt') as f:
    clat = f.read()
    clat= clat[1:-1]
    clat0 = clat.split(",")
    f.close()
with open('clon2.txt') as f:
    clon = f.read()
    clon=clon[1:-1]
    clon0 = clon.split(",")
    f.close()
    
clat,clon=[],[]
for i in range(len(clat0)):
    clat.append(float(clat0[i]))
    clon.append(float(clon0[i]))

#### Definitions

In [17]:
start = datetime(2018, 10, 1) #Start date
# Set Time length [days] and timestep [seconds]
length = 10
duration = timedelta(days=length)
dt = 90 #toggle between - or + to pick backwards or forwards
N = len(clat) # number of deploying locations
n = 1 # 1000   # number of particles per location
dmin = list(np.zeros(len(clat))) #minimum depth
dd = 5 #max depth difference from dmin
x_offset, y_offset, zvals = p_deploy(N,n,dmin,dd)

In [18]:
from parcels import Variable

class MPParticle(JITParticle):        
    ro = Variable('ro', initial = 1025)           
    diameter = Variable('diameter', initial = 1.6e-5)
    length = Variable('length', initial = 61e-5)
    Lb = Variable('Lb', initial = 6)  #days needed in days for particle to have 67% probability of beaching if in beaching zone (500m)
    Db = Variable('Db', initial = 1000) #Distance at which particles can randomly beach.
    Ub = Variable('Ub', initial = 69)  #days to have 67% probability of unbeaching
    sediment = Variable('sediment', initial = 0)
    beached = Variable('beached', initial = 0)

In [19]:
lon = np.zeros([N,n])
lat = np.zeros([N,n])
for i in range(N):
    lon[i,:]=(clon[i] + x_offset[i,:])
    lat[i,:]=(clat[i] + y_offset[i,:])
z = zvals

In [20]:
#Set start date time and the name of the output file
name = 'Beaching-Unbeachingnew2' #name output file
daterange = [start+timedelta(days=i) for i in range(length)]
fn =  name + '_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.nc'
outfile = os.path.join(paths['out'], fn)
print(outfile)

/home/jvalenti/MOAD/results/Beaching-Unbeachingnew220181001_1n_20181011_1n.nc


## Simulation

In [21]:
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W','R']
filenames,variables,dimensions=filename_set(start,length,varlist,local)
field_set=FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)

varlist=['US','VS','WL']
filenames,variables,dimensions=filename_set(start,length,varlist,local)

us = Field.from_netcdf(filenames['US'], variables['US'], dimensions,allow_time_extrapolation=True)
vs = Field.from_netcdf(filenames['VS'], variables['VS'], dimensions,allow_time_extrapolation=True)
wl = Field.from_netcdf(filenames['WL'], variables['WL'], dimensions,allow_time_extrapolation=True)
field_set.add_field(us)
field_set.add_field(vs)
field_set.add_field(wl)
field_set.add_vector_field(VectorField("stokes", us, vs, wl))

filenames,variables,dimensions=filename_set(start,length,['Bathy'],local)
Bth = Field.from_netcdf(filenames['Bathy'], variables['Bathy'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Bth)

# filenames,variables,dimensions=filename_set(start,length,['D'],local)
# Distc = Field.from_netcdf(filenames['D'], variables['D'], dimensions,allow_time_extrapolation=True)
# field_set.add_field(Distc)

In [22]:
# #Load Salish output as fields
#field_set = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)
pset = ParticleSet.from_list(field_set, MPParticle, lon=lon, lat=lat, depth=z, time=start+timedelta(hours=2))

k_sink = pset.Kernel(Buoyancy)
k_waves = pset.Kernel(Stokes_drift)
k_beach = pset.Kernel(Beaching)
k_unbeach = pset.Kernel(Unbeaching)


pset.execute(AdvectionRK4_3D + k_sink + k_waves + k_beach + k_unbeach,
             runtime=duration, 
             dt=dt,
             output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=1)),
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

INFO: Compiled ArrayMPParticleAdvectionRK4_3DBuoyancyStokes_driftBeachingUnbeaching ==> /tmp/parcels-2894/lib946d610e54ba6f74b3c895519e1fddf6_0.so


Particle 1670 lost !! [5400.0, 3.9697923779773325, 50.44062409724402, -126.01156917192925]
Particle 1678 lost !! [5400.0, 4.2793165459691425, 50.46448747328525, -126.1009279237968]
Correct cell not found for (-124.831299, 48.560161) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 429 2
            new particle indices: (yi, xi) 430 44
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) -4.5014257315284496e+01 8.1640877733880191e-01
Particle 1283 lost !! [17820.0, 0.4551173243195096, 48.572911399560304, -124.82141685180402]
Correct cell not found for (-124.802738, 48.525755) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 420 2
            new particle indices: (yi, xi) 422 44
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) -4.5062242238420161e+01 1.6932054322103976e-01
Particle 1268 lost !! [18360.0, 1.2845967449758715, 48.53848703337114, -124.7928684540224]
Correct cell 

INFO: Temporary output files are stored in /home/jvalenti/MOAD/results/out-RPGGNPLK.
INFO: You can use "parcels_convert_npydir_to_netcdf /home/jvalenti/MOAD/results/out-RPGGNPLK" to convert these to a NetCDF file during the run.
  2% (19800.0 of 864000.0) |             | Elapsed Time: 0:00:02 ETA:   0:01:32

Correct cell not found for (-124.742705, 48.454263) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 402 2
            new particle indices: (yi, xi) 404 44
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) -4.5084769938543218e+01 2.3042181769761863e-01
Particle 1240 lost !! [25200.0, 1.7635703371940383, 48.467233168836756, -124.73294945495545]


  2% (21600.0 of 864000.0) |             | Elapsed Time: 0:00:02 ETA:   0:09:17

...

 96% (829800.0 of 864000.0) |########### | Elapsed Time: 0:09:21 ETA:   0:00:38

Correct cell not found for (-124.804795, 48.528589) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 421 2
            new particle indices: (yi, xi) 422 44
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) -4.5013773999173750e+01 8.6010686593563157e-01
Particle 976 lost !! [835740.0, 20.300271895591948, 48.54150340811526, -124.79503769852221]


 96% (835200.0 of 864000.0) |########### | Elapsed Time: 0:09:24 ETA:   0:00:22

Correct cell not found for (-124.756196, 48.470668) after 1000000 iterations
Debug info: old particle indices: (yi, xi) 407 2
            new particle indices: (yi, xi) 408 44
            Mesh 2d shape:  898 398
            Relative particle position:  (xsi, eta) -4.5048728218268174e+01 3.1878555517877677e-01
Particle 1127 lost !! [842220.0, 26.864153856137293, 48.483821676498735, -124.74684687014705]


100% (864000.0 of 864000.0) |############| Elapsed Time: 0:09:37 Time:  0:09:37
/home/jvalenti/conda_envs/parcels/lib/python3.9/site-packages/numpy/lib/arraysetops.py:270: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)
